In [ ]:
# Reading data from sale_data.dat file

with open('sale_data.dat') as f:
    data: list[set[int]] = [{int(i) for i in b.split()} for b in f.readlines()]

n_baskets = len(data)
print(f'The number of baskets is: {n_baskets}')

print(data[:4])


In [ ]:
import numpy as np
from apriori import A_Priori

In [ ]:
threshold = 100
k = 3

frequent_itemsets = A_Priori.get_frequent_itemsets(data, k, threshold)

In [ ]:
for i in range(1, k+1):
    print(f'number of frequent itemsets for entry {i}: {len(frequent_itemsets[i])}')